In [1]:
!pip install gradio langchain chromadb sentence-transformers pypdf transformers accelerate bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.5 MB/s eta 0:

In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [4]:
import os
import gradio as gr
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema.runnable import RunnableMap
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline


In [5]:
def load_pdf_and_embed(pdf_path, persist_directory="db"):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    # Chunk documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(docs)

    # Embed using sentence-transformers
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectordb = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=persist_directory)

    return vectordb


In [6]:
def load_local_phi_model():
    model_id = "microsoft/phi-3-mini-4k-instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True)

    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300, do_sample=True, temperature=0.3)
    llm = HuggingFacePipeline(pipeline=pipe)

    return llm


In [ ]:
def create_qa_chain(vectorstore, llm):
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    # RAG pipeline
    chain = (
        {"context": retriever | RunnablePassthrough(), "question": RunnablePassthrough()}
        | (lambda x: f"Answer the question based on context below.\n\nContext: {x['context']}\n\nQuestion: {x['question']}")
        | llm
        | StrOutputParser()
    )
    return chain


In [7]:
def create_gradio_interface():
    global qa_chain

    def ask_question(query):
        if not qa_chain:
            return "Please upload a PDF first."
        return qa_chain.invoke(query)

    def upload_pdf(file):
        global qa_chain
        vectordb = load_pdf_and_embed(file.name)
        llm = load_local_phi_model()
        qa_chain = create_qa_chain(vectordb, llm)
        return "✅ PDF uploaded and indexed. Ask your questions now."

    with gr.Blocks() as demo:
        gr.Markdown("# 🧠 Local PDF Question Answering\nUpload a PDF and ask questions using Phi-3 model locally.")

        with gr.Row():
            file_input = gr.File(label="Upload PDF")
            upload_btn = gr.Button("Load PDF")
            status = gr.Textbox(label="Status")

        upload_btn.click(upload_pdf, inputs=[file_input], outputs=[status])

        with gr.Row():
            query = gr.Textbox(label="Your Question")
            ask_btn = gr.Button("Ask")
            answer = gr.Textbox(label="Answer")

        ask_btn.click(ask_question, inputs=[query], outputs=[answer])

    return demo


In [8]:
qa_chain = None
demo = create_gradio_interface()
demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c7231f5f6892bda94.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
